# Association rules for online retail data set

In [1]:
#import libraries
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [47]:
# load dataset
dataset = pd.read_excel('Online Retail.xlsx')

In [3]:
# Exploring the dataset: viewing the first few observations
dataset.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
# Exploring the dataset: viewing the last few observations
dataset.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [5]:
# Exploring the dataset: viewing the the number of rows and coulmns for the dataset
dataset.shape

(541909, 8)

In [6]:
# viewing variable datatypes
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
#total number of observations that are null

dataset.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [48]:
# selecting the stockcodes where Description is null

#selects observations where description is null
null_description = dataset[dataset['Description'].isnull()]

#selects stockcodes of the observations where description is null
stockcode= null_description['StockCode']

#taking the indexes of the stackcode series
indexes = stockcode.index


In [49]:
# Selecting description (where available) based on each stockcode

desc = []
# loops through all the elements of the stockcode
for index in indexes:
    #for a givek stockcode, corresponding description is selected
    x = dataset.loc[dataset['StockCode'] == stockcode[index], 'Description'].to_list()
    # check if description for the a particular stockcode is null
    if x == []:
        #appends the the description
        desc.append(x)
    else:
        # removes null values(selects none null values)
        y = [k for k in x if k != []]
        #appends the first element
        desc.append(y[0])
     

In [50]:
# replacing null Descriptions with the values inferred from the StockCode column

dataset['Description'].mask(dataset['Description'].isnull(), desc, inplace=True)



In [51]:
#checking number of missing values after inferring Descripion from StockCode

dataset.isnull().sum()

InvoiceNo           0
StockCode           0
Description       217
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [52]:
# dropping Description with missing value
dataset.dropna(axis=0, subset=['Description'], inplace=True)

# dropping CustomerID with missing value (since the transaction cannot be attributed to any customer)
dataset.dropna(axis=0, subset=['CustomerID'], inplace=True)

dataset.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [53]:
# removing extra whitespaces (if any) from the Descripion feature
dataset['Description'] = dataset['Description'].str.strip() 

In [54]:
# convertion of the InvoiceNo feature to string to make it suitable for further processing
dataset['InvoiceNo'] = dataset['InvoiceNo'].astype('str')

In [55]:
# removing canceled transactions
dataset = dataset[~dataset['InvoiceNo'].str.contains('C')]

In [80]:
# grouping of the transaction by InvoiceNo and Description to obatin a basket-like dataset suitable for 
#association rule mining

#grouping Description by InvoiceNo so that all products on the same invoiceNo can be collected together
transaction_basket = dataset.groupby(['InvoiceNo', 'Description'])['Quantity'].sum()

#unstacking the grouped data
transaction_basket = transaction_basket.unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [81]:
# Function to assign 1 (product with quantity >=1) or 0 (for product that was not bought)

def product_status(x):
    if x <= 0:
        x = 0
    else:
        x = 1
    return x
basket = transaction_basket.applymap(product_status)
basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [82]:
# removing observations with POSTAGE since they're not part of the products
basket.drop('POSTAGE', inplace=True, axis=1)
basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 DAISY PEGS IN WOOD BOX,12 EGG HOUSE PAINTED WOOD,12 HANGING EGGS HAND PAINTED,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [89]:
# obatining the rules
large_itemset = apriori(basket, min_support=0.02, use_colnames=True)
rules = association_rules(large_itemset, metric='lift', min_threshold=1)
rules.sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
73,(PINK REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",0.029996,0.029186,0.021040,0.701439,24.033032,0.020165,3.251641
72,"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",(PINK REGENCY TEACUP AND SAUCER),0.029186,0.029996,0.021040,0.720887,24.033032,0.020165,3.475313
75,(GREEN REGENCY TEACUP AND SAUCER),"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",0.037279,0.023522,0.021040,0.564399,23.994742,0.020163,2.241683
70,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.023522,0.037279,0.021040,0.894495,23.994742,0.020163,9.124923
9,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.037279,0.029996,0.024817,0.665702,22.193256,0.023698,2.901615
...,...,...,...,...,...,...,...,...,...
20,(JUMBO BAG RED RETROSPOT),(JUMBO SHOPPER VINTAGE RED PAISLEY),0.086319,0.042620,0.021364,0.247500,5.807165,0.017685,1.272266
61,(SPOTTY BUNTING),(PARTY BUNTING),0.054111,0.074450,0.020986,0.387836,5.209375,0.016958,1.511933
60,(PARTY BUNTING),(SPOTTY BUNTING),0.074450,0.054111,0.020986,0.281884,5.209375,0.016958,1.317182
25,(JUMBO BAG RED RETROSPOT),(LUNCH BAG RED RETROSPOT),0.086319,0.069486,0.022928,0.265625,3.822690,0.016930,1.267082


In [90]:
# getting a stronger rule with increased condition
rules[(rules['confidence'] >= .3) & (rules['lift'] >= 3)].sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
73,(PINK REGENCY TEACUP AND SAUCER),"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",0.029996,0.029186,0.021040,0.701439,24.033032,0.020165,3.251641
72,"(ROSES REGENCY TEACUP AND SAUCER, GREEN REGENC...",(PINK REGENCY TEACUP AND SAUCER),0.029186,0.029996,0.021040,0.720887,24.033032,0.020165,3.475313
75,(GREEN REGENCY TEACUP AND SAUCER),"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",0.037279,0.023522,0.021040,0.564399,23.994742,0.020163,2.241683
70,"(PINK REGENCY TEACUP AND SAUCER, ROSES REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.023522,0.037279,0.021040,0.894495,23.994742,0.020163,9.124923
9,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.037279,0.029996,0.024817,0.665702,22.193256,0.023698,2.901615
...,...,...,...,...,...,...,...,...,...
11,(GREEN REGENCY TEACUP AND SAUCER),(REGENCY CAKESTAND 3 TIER),0.037279,0.091929,0.020177,0.541245,5.887623,0.016750,1.979422
67,(ROSES REGENCY TEACUP AND SAUCER),(REGENCY CAKESTAND 3 TIER),0.042242,0.091929,0.022659,0.536398,5.834907,0.018775,1.958731
21,(JUMBO SHOPPER VINTAGE RED PAISLEY),(JUMBO BAG RED RETROSPOT),0.042620,0.086319,0.021364,0.501266,5.807165,0.017685,1.832001
61,(SPOTTY BUNTING),(PARTY BUNTING),0.054111,0.074450,0.020986,0.387836,5.209375,0.016958,1.511933
